<a href="https://colab.research.google.com/github/PDGamerSG/GDSC-Image_Classification/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pdgamersg","key":"35378d4ad319c63c6a143638f767f63c"}'}

In [4]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets list

ref                                                              title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
abdulmalik1518/mobiles-dataset-2025                              Mobiles Dataset (2025)                            20KB  2025-02-18 06:50:24           5999        103  1.0              
asinow/car-price-dataset                                         Car Price Dataset                                135KB  2025-01-26 19:53:28          22763        365  1.0              
mahmoudelhemaly/students-grading-dataset                         Student Performance & Behavior Dataset           508KB  2025-02-17 17:38:46           4850         93  1.0              
samikshadalvi/lungs-diseases-dataset                             Lungs

In [11]:
!kaggle competitions download -c cifar-10

100% 715M/715M [00:33<00:00, 24.1MB/s]
100% 715M/715M [00:33<00:00, 22.7MB/s]
